In [63]:
import pandas as pd
import numpy as np
import geojsonio
from shapely.geometry import mapping, Polygon, shape
import json
import collections
from tqdm import tqdm
import datetime

In [26]:
with open('data/nta.json') as json_file:
    ntas_raw = json.load(json_file)

## Loading Data

### Demographis and weather

In [21]:
demographics = pd.read_csv('data/demographics.csv')

In [ ]:
weather = pd.read_csv('data/weather.csv')

### Green trips

In [ ]:
# load the raw data
trips_green = pd.read_csv('data/green_trips.csv.gz')

In [70]:
# filter, so that the data becomes manageable in size
filter_2014 = pd.to_datetime(trips_green.pickup_datetime) <= datetime.date(2014,8,1)
trips_green_2014 = trips_green.loc[filter_2014,:]
trips_green_2014.shape

(1068602, 9)

### Yellow trips

In [ ]:
# load the raw data
trips_yellow = pd.read_csv('data/yellow_trips.csv.gz')

In [ ]:
# filter, so that the data becomes manageable in size
filter_2014 = pd.to_datetime(trips_yellow.pickup_datetime) <= datetime.date(2014,8,1)
trips_yellow_2014 = trips_yellow.loc[filter_2014,:]
trips_yellow_2014.shape

## Creating a NTA - Trip Mapping

In [41]:
# building a dictionary that contains a shapefile and meta information for every nta
nta_shapefiles = collections.defaultdict(dict)
for nta_district in ntas_raw['features']:
    nta_code = nta_district['properties']['NTACode']
    borough_name = nta_district['properties']['BoroName']
    borough_code = nta_district['properties']['BoroCode']
    geometry = nta_district['geometry']
    shapefile = shape(geometry)
    nta_shapefiles[nta_code] = {'borough_name': borough_name,
                               'borough_code': borough_code,
                               'shapefile':shapefile}

In [51]:
def get_nta(lat, lon, nta_shapefiles):
    point = shape({'coordinates': [lat, lon], 'type':'Point'})
    
    for nta_code, nta_data in nta_shapefiles.items():
        if nta_data['shapefile'].contains(point):
            return nta_code
    return None

In [ ]:
pickup_ntas = []
dropoff_ntas = []

for _, trip_data in tqdm(trips_green_2014.iterrows()):
    pickup_lat = trip_data.pickup_longitude
    pickup_lon = trip_data.pickup_latitude
    dropoff_lat = trip_data.dropoff_longitude
    dropoff_lon = trip_data.dropoff_latitude
    
    pickup_ntas.append(get_nta(pickup_lat, pickup_lon, nta_shapefiles))
    dropoff_ntas.append(get_nta(dropoff_lat, dropoff_lon, nta_shapefiles))
    
trips_green['pickup_nta'] = pickup_ntas
trips_green['dropoff_nta'] = dropoff_ntas


0it [00:00, ?it/s]
1it [00:01,  1.26s/it]
17it [00:01,  1.13it/s]
39it [00:01,  1.61it/s]
58it [00:01,  2.29it/s]
78it [00:01,  3.26it/s]
97it [00:01,  4.62it/s]
126it [00:01,  6.55it/s]
161it [00:01,  9.29it/s]
189it [00:02, 13.08it/s]
215it [00:02, 18.26it/s]
245it [00:02, 25.40it/s]
272it [00:02, 34.77it/s]
304it [00:02, 47.46it/s]
332it [00:02, 60.18it/s]
357it [00:02, 74.92it/s]
380it [00:02, 93.70it/s]
416it [00:03, 120.18it/s]
450it [00:03, 148.61it/s]
479it [00:03, 171.96it/s]
507it [00:03, 176.38it/s]
533it [00:03, 189.60it/s]
560it [00:03, 203.20it/s]
585it [00:03, 210.97it/s]
615it [00:03, 231.58it/s]
642it [00:03, 241.77it/s]
672it [00:04, 256.41it/s]
702it [00:04, 261.75it/s]
732it [00:04, 267.95it/s]
760it [00:04, 264.60it/s]
790it [00:04, 273.97it/s]
821it [00:04, 281.92it/s]
853it [00:04, 290.54it/s]
883it [00:04, 288.46it/s]
916it [00:04, 298.28it/s]
947it [00:04, 296.87it/s]
983it [00:05, 312.41it/s]
1015it [00:05, 314.32it/s]
1050it [00:05, 320.89it/s]
1083it [00:05